In [15]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 13.1 MB/s eta 0:00:00


In [29]:
import numpy as np
import pandas as pd
import optuna

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout,Input

In [17]:
df = pd.read_csv('/content/diabetes.csv')

In [18]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [19]:
# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [20]:
X = df.drop('Outcome',axis=1)
y = df['Outcome']

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.1)

In [22]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [23]:
X_train_scaled

array([[-2.45651169e-01, -1.99340898e+00,  7.87939759e-01, ...,
         2.91946115e-01, -6.84507866e-01,  1.05619626e+00],
       [-5.42408088e-01, -3.36694024e-02, -1.50612355e+00, ...,
        -8.07484743e-01, -6.31487985e-02, -5.36224581e-01],
       [-2.45651169e-01,  1.82642240e+00,  1.11566309e+00, ...,
         1.32817964e-01,  1.98557756e+00,  1.55906599e+00],
       ...,
       [-5.42408088e-01, -1.12979493e+00,  1.82851065e-03, ...,
        -5.03694638e-01,  8.68889802e-01, -7.03847827e-01],
       [-1.13592193e+00, -1.00101252e-01,  9.51801424e-01, ...,
         1.94109240e+00,  2.18221345e-01, -2.00978089e-01],
       [ 5.11057502e-02,  1.16210390e+00,  2.14423931e-01, ...,
         2.30274729e+00, -6.99162561e-01, -1.17166466e-01]])

In [36]:
def build_model(trial, input_dim):
  model = Sequential()
  model.add(Input(shape = (input_dim,)))

  n_layers = trial.suggest_int('n_layers',1,10)

  n_nodes = trial.suggest_int('n_nodes',8,128,step=8)

  active = trial.suggest_categorical('activation',['relu','elu','selu','tanh','sigmoid'])

  dropout = trial.suggest_float('dropout',0.1,0.9)

  for i in range(n_layers):
    model.add(Dense(n_nodes, activation=active))
    model.add(Dropout(rate=dropout))

  model.add(Dense(1,activation='sigmoid'))

  optimizer = trial.suggest_categorical('optimizer',['adam','rmsprop','nadam','adagrad'])

  model.compile(
      optimizer = optimizer,
      loss = 'binary_crossentropy',
      metrics=['accuracy']
  )

  return model

In [37]:
def objective(trial):

  tensorflow.keras.backend.clear_session()

  model = build_model(trial,input_dim=X_train_scaled.shape[1])

  batch_size = trial.suggest_categorical('batch_size',[50,100,150,200])

  history = model.fit(
      X_train_scaled,
      y_train,
      batch_size = batch_size,
      epochs=30,
      validation_data = (X_test,y_test)
  )

  val_accuracy = max(history.history['val_accuracy'])
  return val_accuracy

In [38]:
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=30)

print("Best trial")
print(" Validation Accuracy:", study.best_value)
print(" Best Hyperparameters:")
for k,v in study.best_params.items():
  print(f"{k}:{v}")

[I 2026-01-03 07:53:36,617] A new study created in memory with name: no-name-2fb91693-a4d2-476c-b9e9-00312fd23ea4


Epoch 1/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 133ms/step - accuracy: 0.5912 - loss: 0.6837 - val_accuracy: 0.4156 - val_loss: 41.5396
Epoch 2/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7292 - loss: 0.5415 - val_accuracy: 0.4156 - val_loss: 50.7195
Epoch 3/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7356 - loss: 0.5359 - val_accuracy: 0.4156 - val_loss: 55.5387
Epoch 4/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7656 - loss: 0.4946 - val_accuracy: 0.4156 - val_loss: 56.8910
Epoch 5/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7318 - loss: 0.5106 - val_accuracy: 0.4156 - val_loss: 60.2792
Epoch 6/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7345 - loss: 0.5291 - val_accuracy: 0.4156 - val_loss: 61.6066
Epoch 7/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7709 - loss: 0.4866 - val_accuracy: 0.4156 - val_loss: 60.5308
Epoch 8/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7516 - loss: 0.5240 - val_accuracy: 0.4

[I 2026-01-03 07:53:47,210] Trial 0 finished with value: 0.41558441519737244 and parameters: {'n_layers': 1, 'n_nodes': 80, 'activation': 'selu', 'dropout': 0.456464689220319, 'optimizer': 'rmsprop', 'batch_size': 50}. Best is trial 0 with value: 0.41558441519737244.


Epoch 1/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 735ms/step - accuracy: 0.5071 - loss: 0.8093 - val_accuracy: 0.4416 - val_loss: 0.7651
Epoch 2/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5648 - loss: 0.7224 - val_accuracy: 0.4416 - val_loss: 0.7728
Epoch 3/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5337 - loss: 0.7509 - val_accuracy: 0.4286 - val_loss: 0.7803
Epoch 4/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5544 - loss: 0.7479 - val_accuracy: 0.4156 - val_loss: 0.7874
Epoch 5/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5635 - loss: 0.7003 - val_accuracy: 0.4156 - val_loss: 0.7944
Epoch 6/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5737 - loss: 0.7096 - val_accuracy: 0.4156 - val_loss: 0.8010
Epoch 7/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5664 - loss: 0.7556 - val_accuracy: 0.4156 - val_loss: 0.8082
Epoch 8/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5797 - loss: 0.7041 - val_accuracy: 0.4156 - val_loss: 0.8157

[I 2026-01-03 07:53:57,507] Trial 1 finished with value: 0.44155845046043396 and parameters: {'n_layers': 5, 'n_nodes': 72, 'activation': 'tanh', 'dropout': 0.42647656998767647, 'optimizer': 'adagrad', 'batch_size': 150}. Best is trial 1 with value: 0.44155845046043396.


...

Epoch 1/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 12s 358ms/step - accuracy: 0.5870 - loss: 6.1218 - val_accuracy: 0.5844 - val_loss: 4.0434
Epoch 2/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5514 - loss: 6.7573 - val_accuracy: 0.5844 - val_loss: 3.5438
Epoch 3/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5605 - loss: 5.6829 - val_accuracy: 0.5844 - val_loss: 2.7090
Epoch 4/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5515 - loss: 5.2315 - val_accuracy: 0.5844 - val_loss: 1.9461
Epoch 5/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5200 - loss: 5.2283 - val_accuracy: 0.5844 - val_loss: 1.3659
Epoch 6/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5239 - loss: 5.3839 - val_accuracy: 0.5844 - val_loss: 1.0413
Epoch 7/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5866 - loss: 3.6114 - val_accuracy: 0.5844 - val_loss: 0.9152
Epoch 8/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5397 - loss: 3.6346 - val_accuracy: 0.5844 - val_lo

[I 2026-01-03 08:00:00,530] Trial 28 finished with value: 0.5844155550003052 and parameters: {'n_layers': 8, 'n_nodes': 88, 'activation': 'elu', 'dropout': 0.775473803627116, 'optimizer': 'nadam', 'batch_size': 50}. Best is trial 26 with value: 0.701298713684082.


Epoch 1/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 13s 417ms/step - accuracy: 0.4940 - loss: 220.3968 - val_accuracy: 0.5844 - val_loss: 2.5971
Epoch 2/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4826 - loss: 193.8356 - val_accuracy: 0.5844 - val_loss: 2.9160
Epoch 3/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4995 - loss: 184.6125 - val_accuracy: 0.5844 - val_loss: 3.0111
Epoch 4/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5437 - loss: 159.2968 - val_accuracy: 0.5844 - val_loss: 2.6573
Epoch 5/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5513 - loss: 123.3709 - val_accuracy: 0.5844 - val_loss: 2.3000
Epoch 6/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5220 - loss: 133.4736 - val_accuracy: 0.5844 - val_loss: 2.0587
Epoch 7/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5411 - loss: 101.6102 - val_accuracy: 0.5844 - val_loss: 1.8606
Epoch 8/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5498 - loss: 92.1466 - val_accuracy:

[I 2026-01-03 08:00:18,362] Trial 29 finished with value: 0.5844155550003052 and parameters: {'n_layers': 9, 'n_nodes': 104, 'activation': 'elu', 'dropout': 0.8962106916131082, 'optimizer': 'nadam', 'batch_size': 50}. Best is trial 26 with value: 0.701298713684082.


Best trial
 Validation Accuracy: 0.701298713684082
 Best Hyperparameters:
n_layers:7
n_nodes:104
activation:elu
dropout:0.8871300206780791
optimizer:nadam
batch_size:50


In [44]:
best_params = study.best_params

final_model = Sequential()

final_model.add(Input(shape = (X_train_scaled.shape[1],)))

for i in range(best_params['n_layers']):
  final_model.add(Dense(best_params['n_nodes'],activation=best_params['activation']))
  final_model.add(Dropout(best_params['dropout']))

final_model.add(Dense(1, activation='sigmoid'))

final_model.compile(
    optimizer = best_params['optimizer'],
    loss = 'binary_crossentropy',
    metrics=['accuracy']
)

final_model.fit(X_train_scaled,y_train, epochs=200, validation_data=(X_test,y_test),callbacks=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True))

Epoch 1/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 12s 185ms/step - accuracy: 0.5019 - loss: 48.8054 - val_accuracy: 0.5844 - val_loss: 1.4321
Epoch 2/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5199 - loss: 39.1483 - val_accuracy: 0.5844 - val_loss: 1.7834
Epoch 3/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5664 - loss: 27.5509 - val_accuracy: 0.5844 - val_loss: 2.5111
Epoch 4/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5035 - loss: 30.0375 - val_accuracy: 0.5844 - val_loss: 2.5107
Epoch 5/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5461 - loss: 28.1241 - val_accuracy: 0.5844 - val_loss: 1.9636
Epoch 6/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4955 - loss: 31.8679 - val_accuracy: 0.5844 - val_loss: 1.5074
Epoch 7/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5646 - loss: 25.9496 - val_accuracy: 0.5844 - val_loss: 1.0366
Epoch 8/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5284 - loss: 19.4759 - val_accuracy: